#Thinkful Data Science Course
##Unit 4: Predicting the Future; Lesson 1: Making Predictions
##Metrics and Cross-Validation

1) Take the loan data you used previously to build your linear regression model.

2) Break the data-set into 10 segments following the example provided here in KFold (http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators).

3) Compute each of the performance metric (MAE, MSE or R2) for all the folds. The average would be the performance of your model.

4) Comment on each of the performance metric you obtained.

###1) Take the loan data you used previously to build your linear regression model.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

# Import data
loansData = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')

In [2]:
# Split into numbers - will return in form [###, ###]
cleanFICORange = loansData['FICO.Range'].map(lambda x: x.split('-'))

In [3]:
# We have a string inside a list. Need to convert each string to integer
# To do this, we use a list comprehension
# We could either choose the first element, last element, or an average value
# The "[0]" at the end makes it so that we only choose the first element
cleanFICOScore = cleanFICORange.map(lambda x: [int(n) for n in x][0])

In [4]:
# According to Thinkful, to understand better, check out: http://carlgroner.me/Python/2011/11/09/An-Introduction-to-List-Comprehensions-in-Python.html

# Assign cleaned score to new column called "FICO.Score"
loansData['FICO.Score'] = cleanFICOScore

In [5]:
# CLEAN INTEREST RATE DATA
clean_ir = loansData["Interest.Rate"].map(lambda x: round(float(x.rstrip("%"))/100, 4))
loansData["Interest.Rate"] = clean_ir

In [6]:
# CLEAN LOAN LENGTH DATA
clean_loanLength = loansData["Loan.Length"].map(lambda x: float(x.rstrip("months")))
loansData["Loan.Length"] = clean_loanLength

####Linear Regression Model

In [7]:
# Linear regression model: 
# InterestRate = b + a1(FICOScore) + a2(LoanAmount)

intrate = loansData['Interest.Rate']
loanamt = loansData['Amount.Requested']
fico = loansData['FICO.Score']

# Create y and x variables
# The dependent variable
y = np.matrix(intrate).transpose()
# The independent variables shaped as columns
x1 = np.matrix(fico).transpose()
x2 = np.matrix(loanamt).transpose()
# put the two columns together to create an input matrix 
x = np.column_stack([x1,x2])

# # Create linear model
# X = sm.add_constant(x)
# model = sm.OLS(y,X)
# f = model.fit()

In [8]:
# coeff = f.params

In [9]:
# # plot:
# line=[]
# line2 = []
# for j in fico:
# 	line.append(coeff[0] + coeff[1]*j + coeff[2]*10000)
# 	line2.append(coeff[0] + coeff[1]*j + coeff[2]*30000)

# plt.close()
# plt.scatter(fico,intrate)
# plt.hold(True)
# plt.plot(fico, line, label = '$10,000 Requested', color = 'blue')
# plt.plot(fico, line2, label = '$30,000 Requested', color = 'green')
# plt.legend(loc = 'upper right')
# plt.ylabel('Interest Rate in %')
# plt.xlabel('FICO Score')
# plt.savefig('Fico_Scatter_10000&30000.png')

# # Load to new CSV file
# # loansData.to_csv('loansData_clean.csv', header=True, index=False)

###2) Break the data-set into 10 segments following the example provided here in KFold

In [10]:
len(fico)

2500

In [12]:
from sklearn import datasets, linear_model, cross_validation, grid_search


###3) Compute each of the performance metric (MAE, MSE or R2) for all the folds. The average would be the performance of your model.

In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print y_test will give a matrix and not an array, but y_predict is an array not a matrix. Should put them in the same format. Used np.squeeze to convert matrix to an array.

fitted y=mx, not y=mx+c

In [19]:
kf = cross_validation.KFold(len(fico), n_folds=10)
r2 = []
mae = []
mse = []
for train_index, test_index in kf:
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = sm.OLS(y_train,X_train)
    f = model.fit()
    y_pred = f.predict(X_test)
    r2.append(r2_score(np.squeeze(np.asarray(y_test)), y_pred))
    mse.append(mean_squared_error(np.squeeze(np.asarray(y_test)), y_pred))
    mae.append(mean_absolute_error(np.squeeze(np.asarray(y_test)), y_pred))



Example of K-Fold from https://www.quantstart.com/articles/Using-Cross-Validation-to-Optimise-a-Machine-Learning-Method-The-Regression-Setting

In [20]:
mae

[0.037454274823712143,
 0.035980360501116332,
 0.034239738510165395,
 0.035191335143468026,
 0.035942188269023047,
 0.036783403783422798,
 0.034645645249887107,
 0.034830174820047799,
 0.037158010056075427,
 0.038255706232854889]

In [23]:
r2

[-0.137745685712789,
 -0.10941226627211109,
 -0.11687879356892616,
 -0.19474821012161403,
 -0.074726482799533533,
 -0.0049900366349580061,
 -0.10622891179000127,
 -0.0033763362585055301,
 -0.076284492992721997,
 -0.16276290004955007]

In [24]:
mse

[0.0020779781640737076,
 0.0018828308590568201,
 0.0017420423631281253,
 0.0018644232736827364,
 0.0019180495589096308,
 0.001906632559453449,
 0.0017210459624910524,
 0.0018042685217776477,
 0.0020065863326843426,
 0.0021483737561197916]

###4) Comment on each of the performance metric you obtained.